<a href="https://colab.research.google.com/github/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/Classifica%C3%A7%C3%A3o_de_Sa%C3%BAde_Fetal_Pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classificação de Saúde Fetal - Pycaret**

<p align=center>
<img src="https://github.com/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/baby.jpg?raw=true" width="50%"></p>
<p align=center>Fonte: Freepik

Esse artigo abordará somente o auto machine learning através da bilioteca Pycaret. Para mais informações sobre a análise de dados e business understanding verifique esse link: 

https://github.com/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/Classifica%C3%A7%C3%A3o_de_Sa%C3%BAde_Fetal.ipynb

A biblioteca Pycaret permite procurar e selecionar o melhor modelo que terá desempenho satisfatório para o nosso dataset. Primerio as bibliotecas serão instaladas e os dados serão carregados. Em seguida importaremos o módulo de classificação, para depois fazer a divisão do dataset entre treino e teste. 

Para achar o melhor modelo, primeiro será feita uma busca com `setup` para obtermos um rank dos melhores modelos e suas métricas. Em seguida selecionaremos o melhor e então fazemos o `tunning` dos seus hiper-parâmetros com posterior análise através de `evaluate_model`. Após essas etapas, é feito predições dos dados de teste com `predict` e então o modelo é finalizado e salvo.

In [1]:
# Importação das bibliotecas necessárias

import pandas as pd

In [2]:
data = 'https://raw.githubusercontent.com/Diego875621/Data_Science/main/Fetal%20Health%20Classification/fetal_health.csv'

df = pd.read_csv(data)
df.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [3]:
# Instalação pycaret
!pip install pycaret -q

     |████████████████████████████████| 320 kB 4.7 MB/s 
     |████████████████████████████████| 1.3 MB 53.6 MB/s 
     |████████████████████████████████| 25.9 MB 1.7 MB/s 
     |████████████████████████████████| 88 kB 6.8 MB/s 
     |████████████████████████████████| 6.8 MB 16.1 MB/s 
     |████████████████████████████████| 3.3 MB 36.2 MB/s 
     |████████████████████████████████| 17.0 MB 39.1 MB/s 
     |████████████████████████████████| 315 kB 46.5 MB/s 
     |████████████████████████████████| 636 kB 53.4 MB/s 
     |████████████████████████████████| 4.8 MB 29.6 MB/s 
     |████████████████████████████████| 2.0 MB 42.7 MB/s 
     |████████████████████████████████| 1.7 MB 47.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 141 kB 63.3 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |█████████████████

In [21]:
# Desabilitar mensagens de log
import logging, sys
logging.disable(sys.maxsize)

# Habilitar o Colab mode
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [22]:
# Módulos de classificação]
import numpy as np
from pycaret.classification import *

In [23]:
# Divisão treino x test
test = df.sample(frac = 0.25)
train = df.drop(test.index)

test.reset_index(inplace = True, drop = True)
train.reset_index(inplace = True, drop = True)

In [24]:
# Setup
clf = setup(data = df, target = 'fetal_health', fix_imbalance=True, remove_outliers=True,
            log_experiment = True, experiment_name = 'clf01')

,Description,Value
0,session_id,5341
1,Target,fetal_health
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(2126, 22)"
5,Missing Values,False
6,Numeric Features,20
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


PicklingError: ignored

In [25]:
# Comparação entre os modelos
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9498,0.9868,0.9033,0.9506,0.9494,0.8544,0.8554,1.107
et,Extra Trees Classifier,0.9412,0.9835,0.8870,0.9448,0.9416,0.8322,0.8344,0.575
rf,Random Forest Classifier,0.9399,0.9849,0.8992,0.9433,0.9404,0.8296,0.8314,0.996
gbc,Gradient Boosting Classifier,0.9399,0.9789,0.8970,0.9427,0.9401,0.8286,0.8305,2.956
dt,Decision Tree Classifier,0.9002,0.8898,0.8452,0.9100,0.9027,0.7276,0.7317,0.052
ada,Ada Boost Classifier,0.8655,0.8671,0.7754,0.8814,0.8697,0.6355,0.6406,0.390
knn,K Neighbors Classifier,0.8619,0.9229,0.8441,0.8915,0.8707,0.6523,0.6656,0.135
lda,Linear Discriminant Analysis,0.8309,0.9570,0.8126,0.8913,0.8477,0.6091,0.6369,0.036
lr,Logistic Regression,0.8125,0.9302,0.7933,0.8711,0.8302,0.5628,0.5858,1.915
ridge,Ridge Classifier,0.8033,0.0000,0.7953,0.8697,0.8229,0.5494,0.5760,0.029


In [26]:
# Melhor modelo
best

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5341, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [27]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9577,0.9894,0.9352,0.9593,0.9582,0.8830,0.8837
1,0.9437,0.9854,0.9246,0.9437,0.9437,0.8400,0.8400
2,0.9296,0.9804,0.8376,0.9279,0.9284,0.7943,0.7950
3,0.9504,0.9903,0.9018,0.9517,0.9480,0.8501,0.8533
4,0.9645,0.9935,0.9335,0.9663,0.9640,0.8958,0.8968
5,0.9433,0.9853,0.8601,0.9463,0.9435,0.8343,0.8348
6,0.9291,0.9776,0.8413,0.9277,0.9281,0.7876,0.7883
7,0.9504,0.9840,0.9276,0.9535,0.9512,0.8611,0.8628
8,0.9645,0.9921,0.9592,0.9654,0.9649,0.9011,0.9013


In [28]:
# Tunning
tuned_lightgbm = tune_model(lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9225,0.9709,0.8816,0.9236,0.9229,0.7831,0.7833
1,0.9577,0.9787,0.9435,0.9577,0.9577,0.8800,0.8800
2,0.9296,0.9820,0.8505,0.9296,0.9296,0.7999,0.7999
3,0.9504,0.9865,0.9276,0.9545,0.9509,0.8582,0.8592
4,0.9504,0.9785,0.9147,0.9510,0.9493,0.8541,0.8551
5,0.9362,0.9843,0.8571,0.9413,0.9372,0.8160,0.8169
6,0.9291,0.9691,0.8413,0.9310,0.9289,0.7900,0.7905
7,0.9433,0.9828,0.9375,0.9523,0.9451,0.8471,0.8533
8,0.9574,0.9943,0.9691,0.9614,0.9584,0.8858,0.8884


In [29]:
# Comparação dos hiper-parâmetros antes e pós tunning
print(lightgbm)

print('\n\n')

print(' GBR após Tunning: \n', tuned_lightgbm)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5341, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)



 GBR após Tunning: 
 LGBMClassifier(bagging_fraction=0.4, bagging_freq=4, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.3, max_depth=-1,
               min_child_samples=26, min_child_weight=0.001, min_split_gain=0.6,
               n_estimators=150, n_jobs=-1, num_leaves=256, objective=None,
               random_state=5341, reg_alpha=0.5, reg_lambda=0.005,
               silent='warn', subsample=1.0, subsample_for_bin=20

In [30]:
# Avaliação do modelo
evaluate_model(tuned_lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [31]:
final_lightgbm = finalize_model(tuned_lightgbm)

In [32]:
final_lightgbm

LGBMClassifier(bagging_fraction=0.4, bagging_freq=4, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.3, max_depth=-1,
               min_child_samples=26, min_child_weight=0.001, min_split_gain=0.6,
               n_estimators=150, n_jobs=-1, num_leaves=256, objective=None,
               random_state=5341, reg_alpha=0.5, reg_lambda=0.005,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [33]:
predict_model(final_lightgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9812,0.9982,0.9851,0.9823,0.9815,0.9455,0.9463


IndexError: ignored

In [34]:
test_predict = predict_model(final_lightgbm, data = test)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9774,0.9982,0.9777,0.9785,0.9777,0.9389,0.9394


IndexError: ignored

In [35]:
save_model(final_lightgbm, 'lightgbm_classifier_06_11_2022')

PicklingError: ignored

O modelo selecionado pelo Pycaret como melhor foi o LightGBM, atingindo as seguintes métricas: 
- F1 score = 0.9777
- AUC = 0.9982
- Kappa score = 0.9455

Segundo a matrix de confusão, foi obtido um recall = 95,7% para a classe patológica, indicando bons resultados considerando todas as métricas e um modelo de uso satisfatório.

Para ter acesso a discussões e conclusões da análise desse dataset, acesse:

https://github.com/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/Classifica%C3%A7%C3%A3o_de_Sa%C3%BAde_Fetal.ipynb

Imagem:

<a href="https://www.freepik.com/free-vector/baby-birth-concept-illustration_21532448.htm#query=fetus&position=12&from_view=search&track=sph">Image by storyset</a> on Freepik